# Tests for `load_data` 
This notebook serves to address [this](https://github.com/ciemss/pyciemss/issues/434) Github issue.

The new interface for calibrate requires csv files, but there are lots of ways that csv files can fail to provide information in the right format. Test the most common failure modes, such as:

1. Missing data
2. Incorrectly typed columns
3. Mislabeled columns
4. Header columns have one fewer column than data
5. Alignment issues
6. Escaping commas
7. Na, NaN, None, '',,

All of these issues will make it difficult to convert a dataframe to a correctly typed tensor.

### Load dependencies

In [1]:
import os
import pyciemss
from pyciemss.interfaces import calibrate

### Load models

In [2]:
MODEL_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/"

# Petrinets
petri1 = os.path.join(MODEL_PATH, "SEIRD_base_model01_petrinet.json")
petri2 = os.path.join(MODEL_PATH, "SEIRHD_base_model01_petrinet.json")
petri3 = os.path.join(MODEL_PATH, "SEIRHD_with_reinfection01_petrinet.json")
petri4 = os.path.join(MODEL_PATH, "SEIRHD_NPI_Type1_petrinet.json")
petri5 = os.path.join(MODEL_PATH, "SEIRHD_NPI_Type2_petrinet.json")

# Regnets
regnet1 = os.path.join(MODEL_PATH, "LV_goat_chupacabra_regnet.json")
regnet2 = os.path.join(MODEL_PATH, "LV_sheep_foxes_regnet.json")
regnet3 = os.path.join(MODEL_PATH, "LV_rabbits_wolves_regnet.json")
regnet4 = os.path.join(MODEL_PATH, "LV_rabbits_wolves_model02_regnet.json")
regnet5 = os.path.join(MODEL_PATH, "LV_rabbits_wolves_model03_regnet.json")

# Stock-and-Flow
stock1 = os.path.join(MODEL_PATH, "SIR_stockflow.json")
stock2 = os.path.join(MODEL_PATH, "SEIR_stockflow.json")
stock3 = os.path.join(MODEL_PATH, "SEIRD_stockflow.json")
stock4 = os.path.join(MODEL_PATH, "SEIRHD_stockflow.json")
stock5 = os.path.join(MODEL_PATH, "SEIRHDS_stockflow.json")

### Load datasets

In [3]:
dataset1 = "../../../docs/source/sa-testing-notebooks/SIR_data_case_hosp.csv" # this dataset should work fine
# dataset2 = "../../../docs/source/sa-testing-notebooks/SIR_missing_data.csv"
# dataset3 = "../../../docs/source/sa-testing-notebooks/SIR_data_nan.csv"
# dataset4 = "../../../docs/source/sa-testing-notebooks/SIR_missing_data_space.csv"
# dataset5 = "../../../docs/source/sa-testing-notebooks/SIR_data_none_type.csv"

### Calibrate models to data

### Calibrate works when data is mapped to STATE VARIABLES...

In [ ]:
num_iterations = 10

In [ ]:
data_mapping = {"case": "I", "hosp": "H"} # data_mapping = "column_name": "observable/state_variable"
calibrated_results = calibrate(petri2, dataset1, data_mapping=data_mapping, num_iterations=num_iterations)
parameter_estimates = calibrated_results[0]
calibrated_results

In [ ]:
point_estimates = calibrated_results[0](0)
point_estimates

In [ ]:
gaussian_estimates = calibrated_results[0](1)
gaussian_estimates

### Pass inferred parameters to sample 

In [5]:
start_time = 0.0
end_time = 100.
logging_step_size = 10.0
num_samples = 3

# pyciemss.sample(petri2, end_time, logging_step_size, num_samples, 
#                 start_time=start_time, inferred_parameters=parameter_estimates)

In [6]:
pyciemss.sample(petri2, end_time, logging_step_size, num_samples, 
                start_time=start_time)

{'data':     timepoint_id  sample_id  persistent_beta_param  persistent_gamma_param  \
 0              0          0               0.356015                0.229369   
 1              1          0               0.356015                0.229369   
 2              2          0               0.356015                0.229369   
 3              3          0               0.356015                0.229369   
 4              4          0               0.356015                0.229369   
 5              5          0               0.356015                0.229369   
 6              6          0               0.356015                0.229369   
 7              7          0               0.356015                0.229369   
 8              8          0               0.356015                0.229369   
 9              0          1               0.244306                0.334480   
 10             1          1               0.244306                0.334480   
 11             2          1               0

### ...but NOT when data is mapped to OBSERVABLES

In [ ]:
# Try with infected_observable 
data_mapping = {"hosp": "hospitalized_observable", "case": "infected_observable"}
results = calibrate(petri2, dataset2, data_mapping=data_mapping)
results

In [ ]:
data_mapping = {"case": "infected", "hosp": "hospitalized"}
results = calibrate(petri2, dataset2, data_mapping=data_mapping)
results

### (1) Missing data

In [ ]:
# Calibrate fails when there is missing data (an empty string) in the dataset
data_mapping = {"case": "I", "hosp": "H"} # data_mapping = "column_name": "observable/state_variable"
calibrated_results = calibrate(petri2, dataset2, data_mapping=data_mapping)
parameter_estimates = calibrated_results[0]
calibrated_results

In [ ]:
# Calibrate fails when there is a NaN value in the dataset
data_mapping = {"case": "I", "hosp": "H"} # data_mapping = "column_name": "observable/state_variable"
calibrated_results = calibrate(petri2, dataset3, data_mapping=data_mapping)
parameter_estimates = calibrated_results[0]
calibrated_results

In [ ]:
# Calibrate fails when there is a space ' ' in the dataset
data_mapping = {"case": "I", "hosp": "H"} # data_mapping = "column_name": "observable/state_variable"
calibrated_results = calibrate(petri2, dataset4, data_mapping=data_mapping)
parameter_estimates = calibrated_results[0]
calibrated_results

In [ ]:
# Calibrate fails when there is None type in the dataset
data_mapping = {"case": "I", "hosp": "H"} # data_mapping = "column_name": "observable/state_variable"
calibrated_results = calibrate(petri2, dataset5, data_mapping=data_mapping)
parameter_estimates = calibrated_results[0]
calibrated_results

### (2) Incorrectly typed columns

### (3) Mislabeled columns

### (4) Header columns have one fewer column than data

### (5) Other alignment issues

### (6) Escaping commas

### (7) Na, NaN, None, empty string